In [55]:
sys.path.append("..")
import os
import utils
import numpy as np
import pandas as pd

In [58]:
def read_all_results(dir_path):
    paths = os.listdir(dir_path)
    try: paths.remove('archive') 
    except: pass
    last_gens = []
    for i in range(len(paths)):
        path = dir_path + paths[i]
        last_gen = sorted(map(lambda x: x.split("_"), 
                        os.listdir(f"{path}/individuals/")), 
                        key=lambda x: int(x[1].split(".")[0]))[-1][-1][:-4]
        print(f"Path: {paths[i]}, last generation: {int(last_gen)}")
        last_gens.append(int(last_gen))
    return paths, last_gens

In [59]:
dir_path = "../results/"
paths, last_gens = read_all_results(dir_path)

Path: GA_2021_05_14_13_38_13, last generation: 31
Path: GA_2021_05_14_13_37_09, last generation: 29
Path: GA_2021_05_14_13_39_01, last generation: 29
Path: GA_2021_05_14_13_37_48, last generation: 31
Path: GA_2021_05_14_13_38_36, last generation: 32
Path: GA_2021_05_14_13_37_33, last generation: 31


In [75]:
run = 2
gen = 29
path = dir_path + paths[run]

individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
score_path       =  f"{path}/final_scores/final_score_{gen}.pkl"
overview_path    =  f"{path}/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [76]:
with open(dir_path + paths[run] + "/run_params.json", "r") as file:
    print(file.read())
    file.close()

{
    "objective": "yll",
    "simulations": 20,
    "process": {
        "horizon": 79,
        "decision_period": 28,
        "policy": "weighted"
    },
    "population_size": 40,
    "random_individuals": true,
    "min_generations": 40
}


In [77]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
for ix, i in enumerate(sorted(individuals, key=lambda x: x.mean_score)):
    if ix == 0:
        print("Strategy count:")
        print(f"{'U':>5}{'D':>5}{'N':>5}")
        for w in range(1,4):
            print(f"{w}: {i.strategy_count[gen]['U'][w]:<5}{i.strategy_count[gen]['D'][w]:<5}{i.strategy_count[gen]['N'][w]:<5}")
        print()
    print(f"{utils.tcolors.BOLD}{ordinal(ix+1)} place:{utils.tcolors.ENDC}")
    print(f"ID: {i.ID}")
    print(f"Score: {np.round(i.mean_score, 3)}")
    
    print(f"Genes:\n{np.round(i.genes, 3)}")
    print()

Strategy count:
    U    D    N
1: 18   18   11   
2: 5    9    7    
3: 0    1    0    

1st place:
ID: gen_22_013
Score: 53069.05
Genes:
[[[0.    0.    0.    1.    0.    0.   ]
  [0.225 0.15  0.    0.    0.537 0.087]
  [0.    0.    0.    1.    0.    0.   ]]

 [[0.    0.    0.5   0.5   0.    0.   ]
  [0.5   0.    0.    0.    0.5   0.   ]
  [0.    0.    0.    0.879 0.    0.121]]

 [[0.242 0.    0.    0.111 0.426 0.221]
  [0.591 0.295 0.    0.114 0.    0.   ]
  [0.    0.31  0.    0.364 0.25  0.076]]]

2nd place:
ID: gen_24_013
Score: 53103.95
Genes:
[[[0.    0.    0.    1.    0.    0.   ]
  [0.271 0.181 0.    0.    0.432 0.115]
  [0.    0.    0.    1.    0.    0.   ]]

 [[0.    0.    0.625 0.375 0.    0.   ]
  [0.5   0.    0.    0.    0.5   0.   ]
  [0.    0.    0.    0.879 0.    0.121]]

 [[0.242 0.    0.    0.148 0.379 0.231]
  [0.61  0.305 0.    0.086 0.    0.   ]
  [0.    0.233 0.    0.335 0.375 0.057]]]

3rd place:
ID: gen_15_013
Score: 53121.65
Genes:
[[[0.    0.    0.    1.    0.

In [26]:
print(individuals[0].genes, end="\n"*2)
genes = np.sum(individuals[0].genes, axis=(0,1))
norm = np.sum(genes)
norm_genes = np.divide(genes, norm)
print(norm_genes)

[[[0.08571429 0.         0.05714286 0.85714286]
  [0.         0.         0.37860804 0.62139196]
  [0.1119403  0.         0.         0.8880597 ]]

 [[0.         0.05666848 0.5984192  0.34491232]
  [0.         0.25       0.0625     0.6875    ]
  [0.         0.         0.         1.        ]]

 [[0.11842008 0.         0.54769288 0.33388704]
  [0.02637035 0.         0.18581543 0.78781422]
  [0.11864407 0.         0.25       0.63135593]]]

[0.05123212 0.03407428 0.23113093 0.68356267]


In [79]:
from vaccine_allocation_model.MDP import MarkovDecisionProcess
def get_objective(self, objective):
        return {"deaths": lambda process: np.sum(process.state.D),
                "weighted": lambda process: np.sum(process.state.total_infected)*0.01 + np.sum(process.state.D),
                "yll": lambda process: calculate_yll(self.expected_years_remaining, process.state.D.sum(axis=0))
                }[objective]


AttributeError: type object 'SimpleGeneticAlgorithm' has no attribute 'get_objective'